In [ ]:
#!pip install -Uqq fastcore --upgrade
#!pip install -Uqq fastai --upgrade

In [ ]:
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master
import timm

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from fastai.vision.all import *

In [ ]:
Labels = pd.read_csv("../input/some-useful-dataframes/img_classes_train.csv").set_index("Image_id")

In [ ]:
img_class = pd.read_csv("../input/some-useful-dataframes/img_classes_train.csv").set_index("Image_id")

In [ ]:
def yippi(x):
    return Labels.loc[str(x).split("/")[-1][:-4]].item()

In [ ]:
k = 0
splits = list(StratifiedKFold(n_splits=5).split(img_class.Label,img_class.Label))
def getsplit(x):
    return splits[k]

In [ ]:
def splitt(x):
    return[list(x.parameters())[:-2],list(x.classifier.parameters())]

In [ ]:
from sklearn.metrics import roc_auc_score,average_precision_score
def Laal(inp,targ):
    return roc_auc_score(torch.nn.functional.one_hot(torch.tensor(to_np(targ))),inp)##########################

In [ ]:
from tqdm.notebook import tqdm,trange

In [ ]:
#aaa
targs = []
preds = []
learners = []

for k in trange(5):

    fields = DataBlock(blocks=(ImageBlock, CategoryBlock),
       get_items=get_image_files,
       get_y=yippi,
       splitter=getsplit,
       item_tfms=RandomResizedCrop(224, min_scale=0.7),
       batch_tfms=aug_transforms())

    dls = fields.dataloaders("../input/siim-covid19-resized-to-256px-jpg/train",bs=16)
    
    #dls.show_batch()
    break
    learn = Learner(dls,timm.create_model('mobilenetv3_large_100', pretrained=True,num_classes=4,drop_rate=0.5),loss_func=CrossEntropyLossFlat(),splitter=splitt,metrics=AccumMetric(Laal,flatten=False,activation="softmax",dim_argmax=1))
    
    #learn.fine_tune(0)
    mixup = MixUp(1.)
    learn.fit_one_cycle(10, cbs=mixup)

    p,t = learn.get_preds()
    targs.append(torch.nn.functional.one_hot(torch.tensor(to_np(t))))
    preds.append(p)
    learners.append(learn)

In [ ]:
#for i,learn in enumerate(learners):
    #learn.export(f"mobile3l{i}.pkl")

In [ ]:
dls.show_batch()

In [ ]:
#fields = DataBlock(blocks=(ImageBlock, CategoryBlock),
       #get_items=get_image_files,
       #get_y=yippi,
       #splitter=getsplit,
       #item_tfms=RandomResizedCrop(512, min_scale=0.7),
       #batch_tfms=aug_transforms())

In [ ]:
#dls = fields.dataloaders("../input/siim-covid19-resized-to-512px-jpg/train",bs=16)

In [ ]:
learn = Learner(dls,timm.create_model('efficientnet_b3', pretrained=True,num_classes=4,drop_rate=0.3),loss_func=CrossEntropyLossFlat(),splitter=splitt,metrics=AccumMetric(Laal,flatten=False,activation="softmax",dim_argmax=1))

In [ ]:
#learn.fine_tune(0)

In [ ]:
learn.lr_find()

In [ ]:
mixup = MixUp(1.)
learn.fit_one_cycle(10,cbs=mixup)

In [ ]:
#learn.fit_one_cycle(3,cbs=mixup)

In [ ]:
learn.recorder.plot_sched() 

In [ ]:
#learn.export("mobil.pkl")